In [1]:
from bs4 import BeautifulSoup as bs
import requests
import json
import re
import time

In [2]:
main_page1= bs(requests.get('https://www.rei.com/c/climbing-shoes?ir=category%3Aclimbing-shoes&pagesize=90').content)
main_page2 = bs(requests.get('https://www.rei.com/c/climbing-shoes?ir=category%3Aclimbing-shoes&pagesize=90&page=2').content)

#### Load up both pages to be scraped for each shoe's url.

In [3]:
shoe_links = main_page1.find_all('a', class_='_1A-arB0CEJjk5iTZIRpjPs')
rel_links = [shoe_links[i]['href'] for i in range(len(shoe_links))]

#### Get all the links for all the shoes on the first page, and save it as a list.

In [5]:
shoe_links1 = main_page2.find_all('a', class_='_1A-arB0CEJjk5iTZIRpjPs')
shoe_links_trans = [shoe_links1[i]['href'] for i in range(len(shoe_links1))]

#### Get all the links for all the shoes on thesecond page, and save it as a list.

In [6]:
rel_links.extend(shoe_links_trans)

In [7]:
len(rel_links)

308

In [8]:
del rel_links[1::2]

#### Get rid of duplicates.

In [9]:
len(rel_links)

154

In [10]:
allowed = re.compile(r'[^^A-Za-z\s\d*\.?\d*$\-]+')
def get_name(link):
    possible = link.split('/')
    actual = possible[-1].replace('-', ' ').title()
    key = 'Name'
    val = actual
    return key, val

#### Function to return the cleaned name of the shoe.

In [18]:
def get_brand(shoe_page):
    brand_name = list(shoe_page.find_all('script',attrs={'data-client-store':"product-details"})[0])[0].split('\n')[15].replace('        ','')
    brand_name = re.sub(allowed, '', brand_name.split(':')[1]).replace(' ', '', 1)
    key = 'Brand'
    val = brand_name
    return key, val

#### Function to return the cleaned brand name of the shoe.

In [19]:
def get_price(shoe_page):
    price_info = list(shoe_page.find_all('script',attrs={'data-client-store':'product-price-data'})[0])[0].split('\n')[4].replace('"','').replace(',','')
    price_info = re.sub(allowed, '', price_info.split(':')[1])
    key = 'Price' 
    val = float(price_info)
    return key, val

#### Function to return a float value for price for each shoe.

In [20]:
def add_to_dict(dict, key, val):
    dict[key] = val
    return dict

#### Function to take the outputs of the previous 3 functions and add them to the dictionary for each shoe properly.

In [21]:
master_list = []
def get_info(url):
    if ('approach') not in url:
        test_features = {}
        shoe_page = bs(requests.get('https://www.rei.com/' + url, 'html5lib').content)
        shoe_details = list(shoe_page.find_all('script',attrs={'data-client-store':"product-details"})[0])[0].split('\n')[3].split('{')
        features = [i.split(',') for i in shoe_details]
        del features[0]
        features = [features[i][j] for i in range(len(features)) for j in range(2)]
        for i in range(0,len(features),2):
            key = features[i].split(':')[1].replace('"','')
            val = re.sub(allowed, '', features[i+1].split(':')[1]).replace('u0027','')
            test_features[key] = val

        add_to_dict(test_features, *(get_price(shoe_page)))

        add_to_dict(test_features, *(get_name(url)))

        add_to_dict(test_features, *(get_brand(shoe_page)))
    
        return test_features
    

#### Main scraping function to create a dictionary for each shoe with 10 features outlined on the website.

In [22]:
get_info(rel_links[15])

{'Best Use': 'Rock Climbing',
 'Climbing Shoe Type': 'Moderate',
 'Last': 'Slip-lasted',
 'Upper': 'Leathersynthetic leather',
 'Lining': 'Pacific forefoot and back',
 'Outsole': '4 mm Vibram XS Edge rubber',
 'Footwear Closure': 'Lace-up',
 'Can Be Resoled': 'Yes',
 'Gender': 'Unisex',
 'Weight (Pair)': '1 lb. 2.42 oz. ounces',
 'Price': 195.0,
 'Name': 'La Sportiva Katana Lace Climbing Shoes',
 'Brand': 'La Sportiva'}

#### Test to ensure function works as intended.

In [31]:
for index, i in enumerate(rel_links[0:137]):
    try:
        master_list.append(get_info(i))
        time.sleep(.15)
    except Exception as e:
        print(index)
        print(e)

#### Passing the scraping function to the list of links, exculding the last 17 due to a difference in their page architecture.

In [32]:
len(master_list)

137

In [34]:
master_list = [i for i in master_list if i]
len(master_list)

110

#### Some pages did not contain rock climbing shoes, but rather approach shoes and returned None in the list, so I removed those entries as they are not needed.

In [45]:
keys = ['Best Use','Climbing Shoe Type','Last','Upper','Lining','Outsole','Footwear Closure','Can Be Resoled',
        'Gender','Weight (Pair)','Price','Name','Brand']
vals = [['Rock Climbing','Moderate','Slip-lasted','Synthetic','Leather','Stealth OnyXX rubber','Strap','Yes','Mens',
         '14.2 ounces',86.73,'Five Ten Arrowhead Mens','Five Ten'],
        ['Rock Climbing','Moderate','Slip-lasted','Leather','Clarino synthetic leather','Stealth C4 rubber','Strap','Yes',
        'Mens','1 lb 2.9 oz.',59.73,'Five Ten Verdon VCS Mens','Five Ten'],
        ['Rock Climbing','Neutral','Slip-lasted','Polyester/leather','Synthetic','Stealth OnyXX rubber','Lace-up','Yes',
         'Womens','12.6 ounces',71.73,'Five Ten Siren Womens','Five Ten'],
        ['Rock Climbing','Aggressive','Slip-lasted''Synthetic leather','Cotton canvas','Dark Matter rubber','Strap','Yes',
         'Womens','16 ounces',99.73,'Free Range LV Womens','So iLL'],
        ['Rock Climbing','Neutral','Slip-lasted','Leather','Cotton','Stealth® C4 rubber','Strap','Yes','Womens',
         '15.2 ounces',47.73,'Five Ten Gambit VCS Womens','Five Ten']]
def add_dict(a,b):
    master_list.append(dict(zip(a,b)))

In [47]:
for i in vals:
    add_dict(keys,i)

In [48]:
len(master_list)

115

#### Looking at the data there were 5 shoes with different page architectures which were handed manually as it was a low number of entries. This add_dict function just appends these entries to the master list, and we can see they have successfully been added.

In [49]:
master_list[0:50]

[{'Best Use': 'Rock Climbing',
  'Climbing Shoe Type': 'Neutral',
  'Last': 'Slip-lasted',
  'Upper': 'Synthetic knit',
  'Lining': 'Hemp',
  'Outsole': 'Rubber',
  'Footwear Closure': 'Strap',
  'Can Be Resoled': 'Yes',
  'Gender': 'Mens',
  'Weight (Pair)': '15.6 ounces',
  'Price': 69.93,
  'Name': 'Black Diamond Momentum Climbing Shoes Ash Mens',
  'Brand': 'Black Diamond'},
 {'Best Use': 'Rock Climbing',
  'Climbing Shoe Type': 'Neutral',
  'Last': 'Slip-lasted',
  'Upper': 'Knit',
  'Lining': 'Hemp',
  'Outsole': 'Rubber',
  'Footwear Closure': 'Strap',
  'Can Be Resoled': 'Yes',
  'Vegan': 'Yes',
  'Gender': 'Mens',
  'Weight (Pair)': 'Unavailable',
  'Price': 59.93,
  'Name': 'Black Diamond Momentum Vegan Climbing Shoes Mens',
  'Brand': 'Black Diamond'},
 {'Best Use': 'Climbing',
  'Climbing Shoe Type': 'Neutral',
  'Last': 'Slip-lasted',
  'Upper': 'Synthetic',
  'Lining': 'AirTex Silver',
  'Outsole': '5mm FriXion RS',
  'Footwear Closure': 'Strap',
  'Can Be Resoled': 'Yes'

In [50]:
def save_data(file,data):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

#### This function save the list of dictionaries as a json file which can later be loaded in and easily transformed into a Pandas Dataframe for exploration and modelling.